### Following thigns are to be done and encapsulated in this project  

##### 1) Importing Library  

##### 2) Build Class 'Model' 

Model class will have all the things that is my network , input , output , all components of forward propagation and all components of backward propagation 

##### 3) Importing Image 
So for this again I'll build a function 

##### 4) Building model 

##### 5) Output 

### Importing Library  

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline 

In [3]:
import math 
from random import random 

In [4]:
from PIL import Image 
import pandas as pd 

In [5]:
import copy 

### Building Class Model  

In [27]:
class Model() :
    network = [[1 for i in range(4)] for j in range(100000)]
    it =  0 

    def input(self,input) :
        self.input = input 
        
    def output(self,output) :
        self.output = output 
        
    def add(self,input_shape , prev_shape , activation_func) :
            weight = [[random() for i in range(prev_shape)] for j in range(input_shape)]
            neuron = [random() for i in range(input_shape)]
            bias = random()

            self.network[self.it][0],self.network[self.it][1],self.network[self.it][2],self.network[self.it][3] =  weight,neuron,bias,activation_func
            self.it+=1 
            
    
            
    
    

### Building all components of Forward Propagation  

In [7]:
def active_sum(input,weight,bias) :
    activation = bias 
    for i in range(len(weight)) : 
        activation += (weight[i]*input[i]) 
        
    return activation

In [9]:
def sigmoid(input) :
    return 1.0/(1.0 + np.exp(-input))

In [8]:
def ReLU(input) :
    if(input >0) :
        return input 
    return (0.01*input)

In [10]:
def Activation(sum_active,function) :
    if(function == 'ReLU'):
        return ReLU(sum_active)
    if(function == 'sigmoid') :
        return sigmoid(sum_active)

In [11]:
def forward_prop(network,input,bias,it) :
    #print(it)
    my_input = input
    my_bias = bias 
    for i in range(it-1) :  #Each hidden layer
        for j in range(len(network[i][1])): #For each neuron 
            sum_active = active_sum(network[i][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[i][3])
            #print(activation)           
            network[i][1][j] = activation
            
        my_input = network[i][1]
        my_bias = network[i][2]
        
    for j in range(len(network[it-1][1])): #For each neuron 
            sum_active = active_sum(network[it-1][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[it-1][3])      
            network[it-1][1][j] = activation
                       
    return network

### Building all components of Backward Propagation  

In [12]:
def sig_derivative(input) :
    
    return sigmoid(input)*(1.0  - sigmoid(input))

In [13]:
def rel_derivative(input) :
    if(input >0) :
        return 1 
    return (-0.01) 

In [60]:
def derivative(input,function) :
    if(function == 'ReLU') :
       ## print("ReLu")
        return rel_derivative(input)
    if(function == 'sigmoid') :
        #print("Sigmoid")
        return sig_derivative(input)

In [61]:
def back_prop(network,input,output,learning_rate,bias,it) :
    error = [[1 for i in range(1000)] for j in range(it)]
    #For last layer 
    it-=1 
    sum_det = 0 #For bias 
    for i in range(len(network[it][1])) : #For each neuron 
        
        error[it][i] = output[i] - network[it][1][i]  #Error of ith output neuron
        #print("error in ",it," layer and ",i,"th neuron is ",error[it][i])            #Printing !!!!!!!!!!
        delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
        sum_det += delta
        error[it][i] = delta 
        
        #Now adjusting all weights to this neuron 
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
    network[it-1][2] *= (1 + learning_rate*sum_det)
    
    it-=1 
    
    
    ## Now doing for all layers but first 
    while(it != 0) :
        sum_det = 0
        for i in range(len(network[it][1])) : #For each neuron 
            
            err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])
            
            for j in range(len(network[it+1][1])) :
                err += (error[it+1][j]*network[it+1][0][j][i])
                
            
            error[it][i] = err 
            delta = error[it][i]*derivative(network[it][1][i],network[it][3]) 
            error[it][i] = delta
            sum_det += delta
            for j in range(len(network[it][0][i])) :
                network[it][0][i][j] += (learning_rate*delta*network[it-1][1][j])
            
            
        network[it-1][2] *= (1 + learning_rate*sum_det)

        it-=1 
        
        
    #Now for last layer . Here the only change will be that instead of network we'll take input and input bias 
    
    sum_det = 0
    for i in range(len(network[it][1])) : #For each neuron 

        err = 0 #Calculating err for each neuron which is summation(error[it+1][j]*w[j][i])

        for j in range(len(network[it+1][1])) :
            err += (error[it+1][j]*network[it+1][0][j][i])


        error[it][i] = err 
        delta = error[it][i]*derivative(network[it][1][i],network[it][3])  
        error[it][i] = delta
        sum_det += delta
        for j in range(len(network[it][0][i])) :
            network[it][0][i][j] += (learning_rate*delta*input[j])


    bias*= (1 + learning_rate*sum_det)
    
            
            
    
    return network
        
    

### Writing train and predict function  

In [82]:
def train(network,input,output,bias,learning_rate,it,epoch) :
    leng = len(network)
    for i in range(epoch) :
        network = forward_prop(network,input,bias,it) 
        network = back_prop(network,input,output,learning_rate,bias,it) 
    
    return network

In [90]:
def predict(input,network,bias,it) :
    dict_answer = ['J','P','G']
    my_input = input
    my_bias = bias 
    for i in range(it-1) :  #Each hidden layer
        for j in range(len(network[i][1])): #For each neuron 
            sum_active = active_sum(network[i][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[i][3])      
            network[i][1][j] = activation
            
        my_input = network[i][1]
        my_bias = network[i][2]
        
    for j in range(len(network[it-1][1])): #For each neuron 
            sum_active = active_sum(network[it-1][0][j],my_input,my_bias)
            activation = Activation(sum_active,network[it-1][3])         
            network[it-1][1][j] = activation
            
    #return network[it-1][1]
    
    max = 1 
    answer = network[it-1][1] 
    if(answer[0] > answer[1]) :
        max = 0
    if(answer[max] < answer[2]) :
        max = 2 
        
    return dict_answer[max]
    
    

#### Function to take image as input ,resize to 10X10 and convert to 'L' and then return pixels  

In [18]:
def image_input(file) :
    img = Image.open(file).resize((10,10)).convert("L")
    pix = list(img.getdata())
    for i in range(len(pix)) :
        if(pix[i] < 255) :
            pix[i]= 1 
        else :
            pix[i] = 0 
            
    return pix 

#### Taking input of my training_data  

In [19]:
j = []
p = []
g = [] 

for i in range(5) :
    file = "J" + str(i+1) + ".png"
    j.append(image_input(file))
    
for i in range(5) :
    file = "P" + str(i+1) + ".png"
    p.append(image_input(file))
    
for i in range(5) :
    file = "G" + str(i+1) + ".png"
    g.append(image_input(file))

#### Building Model

In [73]:
model = Model()

In [74]:
model.add(50,100,'sigmoid')
model.add(3,50,'sigmoid')

In [75]:
bias = 1 
epoch = 25 
learning_rate = 0.1

In [76]:
model.it

2

In [77]:
for i in range(5):
    input = j[i]
    output = [1,0,0]
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
    model.network = train(model.network,input,output,bias,learning_rate,model.it,epoch)
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
    input = p[i]
    output = [0,1,0]
    mean = sum(input) / len(input) 
    variance = sum([((x - mean) ** 2) for x in input]) / len(input)
    for k in range(len(input)) :
        input[k] = (input[k] - mean)/variance
    model.network = train(model.network,input,output,bias,learning_rate,model.it,epoch)
    
    input = g[i]
    output = [0,0,1]
    
    model.network = train(model.network,input,output,bias,learning_rate,model.it,epoch)

In [78]:
model.network[model.it -1][1]

[0.0032715548894102844, 0.08144352703297635, 0.9999389533338721]

In [96]:
tmp = copy.deepcopy(model.network)  
predict(p[2],tmp,bias,model.it)


'P'